In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import pandas as pd
import numpy as np
import scipy.io as scio
from matplotlib import pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.io import loadmat
import scipy

/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Feature Extraction

### Path and Load

In [4]:
class load():
    def load_data(filename):
        raw_data = pd.read_csv(filename)
        raw_data['filename'] = [str(i).zfill(4)+'.jpg' for i in raw_data['Index'].tolist()]
        raw_data['pointsname'] = [str(i).zfill(4)+'.mat' for i in raw_data['Index'].tolist()]
        return raw_data

    def load_points(points_path,data):
        n = data.shape[0]
        points_data = np.zeros([n,3003,2])
        start_time = time.time()
        for i in range(n):
            result = loadmat(points_path+data['pointsname'][i])
            key = sorted(result.keys())[-1]
            points = result[key]
            distance_h = []
            distance_v = []
            for d in range(points.shape[0]-1):
                for j in range(d+1,points.shape[0]):
                    distance_h.append(abs(points[d,0]-points[j,0]))
                    distance_v.append(abs(points[d,1]-points[j,1]))

            points_data[i,:,0]=distance_h
            points_data[i,:,1]=distance_v
        print("--- %s seconds ---" % (time.time() - start_time))
        return points_data.reshape([2500,6006])

In [5]:
path = '/Users/zhaoziqin/Desktop/Applied Data Science/Spring2020-Project3-group5'

trainpath = path+'/data/train_set/'
testpath = path+'/data/test_set/'

train_image = trainpath + "images/"
train_point = trainpath + "points/"

test_image = testpath + "images/"
test_point = testpath + "points/"


### Train Feature Extraction

In [7]:
data = load.load_data(trainpath + 'label.csv')
X = load.load_points(train_point,data)
X = np.round(X,0)
y= data['emotion_idx'].to_numpy()

--- 15.575760841369629 seconds ---


In [8]:
X

array([[ 46.,   4.,  29., ..., 114.,   7.,  57.],
       [ 44.,  12.,  27., ..., 117.,   1.,  59.],
       [ 42.,   5.,  24., ..., 109.,   3.,  54.],
       ...,
       [ 29.,   7.,  16., ...,  89.,   0.,  45.],
       [ 30.,   7.,  16., ..., 101.,   6.,  47.],
       [ 38.,   6.,  17., ...,  99.,   1.,  50.]])

In [9]:
y

array([ 1,  1,  1, ..., 22, 22, 22])

### Test Feature Extraction

In [16]:
n = 2500
points_data = np.zeros([n,3003,2])
start_time = time.time()
for i in range(n):
    result = loadmat(test_point + [str(i+1).zfill(4)+'.mat' for i in range(2500)][i])
    key = sorted(result.keys())[-1]
    points = result[key]
    distance_h = []
    distance_v = []
    for d in range(points.shape[0]-1):
        for j in range(d+1,points.shape[0]):
            distance_h.append(abs(points[d,0]-points[j,0]))
            distance_v.append(abs(points[d,1]-points[j,1]))
    points_data[i,:,0]=distance_h
    points_data[i,:,1]=distance_v
print("--- %s seconds ---" % (time.time() - start_time))

test_x = points_data.reshape([2500,6006])

--- 21.605776071548462 seconds ---


In [17]:
test_x = np.round(test_x,0)
test_x

array([[ 46.,   4.,  29., ..., 114.,   7.,  57.],
       [ 44.,  12.,  27., ..., 117.,   1.,  59.],
       [ 42.,   5.,  24., ..., 109.,   3.,  54.],
       ...,
       [ 29.,   7.,  16., ...,  89.,   0.,  45.],
       [ 30.,   7.,  16., ..., 101.,   6.,  47.],
       [ 38.,   6.,  17., ...,  99.,   1.,  50.]])

In [18]:
test_x == X

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

## Baseline Model --- Training

In [20]:
start_time = time.time()
baseline = GradientBoostingClassifier(n_estimators=100,max_depth= 1,learning_rate=0.1)
gbm_model = baseline.fit(X,y) 
print("--- %s seconds ---" % (time.time() - start_time))

--- 682.0386118888855 seconds ---


In [27]:
joblib.dump(gbm_model,'../output/Baseline_model.m')

['../output/Baseline_model.m']

## Advanced Model --- Training

In [28]:
conda install keras

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |           py37_0         2.8 MB
    openssl-1.1.1d             |       h1de35cc_2         2.2 MB
    ------------------------------------------------------------
                                           Total:         5.0 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  conda                                            anaconda --> pkgs/main
  openssl                                          anaconda --> pkgs/main



openssl-1.1.1d       | 2.2 MB    | ##################################### | 100% 
conda-4.8.3          | 2.8 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

In [29]:
conda install tensorflow

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [30]:
import tensorflow
import keras
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input, Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


#### Model Fitting

In [31]:
Y = to_categorical(y)
Y = Y[:,1:]

In [32]:
input_shape = [6006] 
input_layer = Input(input_shape)
x = BatchNormalization(momentum = 0.88)(input_layer) 
x = Dense(22*10,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
x = Dropout(0.25)(x)
x = BatchNormalization()(x) 
x = Dense(22*8,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
x = Dropout(0.25)(x)
x = Dense(22*4,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
x = Dropout(0.25)(x)
x = Dense(22*2,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
output_layer = Dense(22,activation='softmax',kernel_initializer=initializers.glorot_normal(seed=6))(x) 
model = Model(input_layer,output_layer)

In [33]:
start_time = time.time()
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])
model_nn = model.fit(X,Y,epochs=200)
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/200
2500/2500 [==============================] - 10s 4ms/step - loss: 3.0250 - accuracy: 0.0948
Epoch 2/200
2500/2500 [==============================] - 5s 2ms/step - loss: 2.5314 - accuracy: 0.1936
Epoch 3/200
2500/2500 [==============================] - 5s 2ms/step - loss: 2.2103 - accuracy: 0.2688
Epoch 4/200
2500/2500 [==============================] - 5s 2ms/step - loss: 2.0393 - accuracy: 0.3084
Epoch 5/200
2500/2500 [==============================] - 5s 2ms/step - loss: 1.9423 - accuracy: 0.3404
Epoch 6/200
2500/2500 [==============================] - 5s 2ms/step - loss: 1.8313 - accuracy: 0.3868
Epoch 7/200
2500/2500 [==============================] - 5s 2ms/step - loss: 1.7731 - accuracy: 0.3964
Epoch 8/200
2500/2500 [==============================] - 5s 2ms/step - loss: 1.6873 - accuracy: 0.4224
Epoch 9/200
2500/2500 [==============================] - 5s 2ms/step - loss: 1.6427 - accuracy: 0.4348
Epoch 10/200
2500/2500 [==============================] - 5s 2ms/step - 

2500/2500 [==============================] - 5s 2ms/step - loss: 0.3401 - accuracy: 0.8940
Epoch 158/200
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3190 - accuracy: 0.8984
Epoch 159/200
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3348 - accuracy: 0.8860
Epoch 160/200
2500/2500 [==============================] - 7s 3ms/step - loss: 0.3524 - accuracy: 0.8892
Epoch 161/200
2500/2500 [==============================] - 6s 3ms/step - loss: 0.3544 - accuracy: 0.8828
Epoch 162/200
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3471 - accuracy: 0.8876
Epoch 163/200
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3459 - accuracy: 0.8908
Epoch 164/200
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3634 - accuracy: 0.8820
Epoch 165/200
2500/2500 [==============================] - 5s 2ms/step - loss: 0.3074 - accuracy: 0.9064
Epoch 166/200
2500/2500 [==============================] - 5s 2ms/ste

In [35]:
start_time = time.time()
pred = model.predict(X)
pred_list = [] 
for i in range(len(pred)):
    arr = pred[i]
    idx = np.argwhere(arr == np.max(arr))
    pred_list.append(idx[0][0])
tst_labl = np.argmax(Y, axis=-1)
accuracy = accuracy_score(pred_list, tst_labl)
print("Test accuracy is %s percent" % round(accuracy*100,3))
print("testing model takes %s seconds" % (time.time() - start_time))

Test accuracy is 99.68 percent
testing model takes 1.3371608257293701 seconds


In [36]:
joblib.dump(model,'../output/model_nn.m')

['../output/model_nn.m']

# Prediction

In [13]:
Baseline_model=joblib.load('../output/Baseline_model.m')

Advanced_model=joblib.load('../output/model_nn.m')

Using TensorFlow backend.


In [14]:
Baseline_model

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [15]:
Advanced_model

#### Prediction for Baseline/Advanced model

In [34]:
start_time = time.time()
pred_baseline = Baseline_model.predict(test_x)
print("testing baseline takes %s seconds" % (time.time() - start_time))

testing baseline takes 0.25626707077026367 seconds


In [35]:
start_time = time.time()
pred_advanced  = np.argmax(Advanced_model.predict(test_x), axis=1)+1
print("testing nn takes %s seconds" % (time.time() - start_time))

testing nn takes 1.2242860794067383 seconds


#### Label Prediction

In [33]:
label_pred = pd.read_csv(path+'/data/labels_prediction.csv')
label_pred['Baseline'] = pred_baseline
label_pred['Advanced'] = pred_advanced
label_pred.to_csv(path+'/data/labels_prediction.csv')